In [8]:
import os
import json
import torch
import numpy as np 
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


In [39]:

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)
# train.head(3)

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."


Setting constant variables

In [4]:
EPOCHS = 10
BATCH_SIZE = 200
LR = 0.01
FEATURE_SIZE = 21

One hot encoding function for a given string and its categories(in a list)


In [25]:
def one_hot(categories, string):
    encoding = np.zeros((len(string), len(categories)))
    
    for idx, char in enumerate(string):
        encoding[idx, categories.index(char)] = 1
    return encoding

one hot encoding for one row of train/test, binding and returning; returns a matrix of 107x14

In [31]:
def featurize(entity):
    sequence = one_hot(list('ACGU'), entity['sequence'])
    structure = one_hot(list('.()'), entity['structure'])
    loop_type = one_hot(list('BEHIMSX'), entity['predicted_loop_type'])
    
    features = np.hstack([sequence, structure, loop_type])
    return features

In [45]:
# p = featurize(train.iloc[0])
# len(p)

107

OKay, so this function takes the feature = featurize(string - "AACCGGYUU...") which is a 107x14 matrix
basically what we have to do is consider sub sequences of same length for out dataset, we can take the whole string too, but there is no need for that. so we take the aage k and peeche k 10 elements of string (EG"AACC...") for every element in string and make it the new string. we also add padding incase there arent 10 elements at the back or front in the below funstion.......
finallly we return a 21 length sub-string with/without padding in one hot form (as we took it) ie 21x14 string is returned

In [51]:
def char_encode(index, features, feature_size):
    half_size = (feature_size - 1) // 2
    
    if index - half_size < 0:
        char_features = features[:index+half_size+1]
        padding = np.zeros((int(half_size - index), char_features.shape[1]))
        char_features = np.vstack([padding, char_features])
    elif index + half_size + 1 > len(features):
        char_features = features[index-half_size:]
        padding = np.zeros((int(half_size - (len(features) - index))+1, char_features.shape[1]))
        char_features = np.vstack([char_features, padding])
    else:
        char_features = features[index-half_size:index+half_size+1]
    
    return char_features

In [52]:
class VaxDataset(Dataset):
    def __init__(self, path, test=False):
        self.path = path
        self.test = test
        self.features = []
        self.targets = []
        self.ids = []
        self.load_data()
        
    def load_data(self):
        with open(self.path, 'r') as text:         # read json file into text
            for line in text:                      # gets one row of train/test at a time
                records = json.loads(line)         # that one row is in records now
                features = featurize(records)      # returns a 107x14 matrix --- only 3 rows --- diff for test
                
                for char_i in range(records['seq_scored']):                         # 68 --- diff for test
                    char_features = char_encode(char_i, features, FEATURE_SIZE)     # get 21x14 matrix in char_features
                    self.features.append(char_features)                             # append to N x 21 x 14
                    self.ids.append('%s_%d' % (records['id'], char_i))              #actualId_size
                    
                if not self.test:                  # if its the train set, we need the targets
                    targets = np.stack([records['reactivity'], records['deg_Mg_pH10'], records['deg_Mg_50C']], axis=1)
                    self.targets.extend([targets[char_i] for char_i in range(records['seq_scored'])])
                    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        if self.test:
            return self.features[index], self.ids[index]
        else:
            return self.features[index], self.targets[index], self.ids[index]

    

In [53]:
train_dataset = VaxDataset('../input/stanford-covid-vaccine/train.json')
train_dataloader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

In [56]:
class Flatten(nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)
    
class VaxModel(nn.Module):
    def __init__(self):
        super(VaxModel, self).__init__()     #initializer
        self.layers = nn.Sequential(
            nn.Dropout(0.2),
            nn.Conv1d(14, 32, 1, 1),
            nn.PReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.2),
            nn.Conv1d(32, 1, 1, 1),
            nn.PReLU(),
            Flatten(),
            nn.Dropout(0.2),
            nn.Linear(FEATURE_SIZE, 32),
            nn.PReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.2),
            nn.Linear(32, 3),
        )
        
    def forward(self, features):
        return self.layers(features)

In [57]:
model = VaxModel()
optimizer = torch.optim.Adam(model.parameters(), LR)
criterion = nn.MSELoss()

In [58]:
for epoch in range(EPOCHS):
    losses = []
    model.train()
    for features, targets, ids in train_dataloader:
        
        features = features.permute(0,2,1).float()
        
        targets  = targets.float()
        predictions = model(features)
        loss = criterion(predictions, targets)
        
        for p in model.parameters():
            p.grad = None
        
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().numpy())
    
    avg_loss = float(np.mean(losses))
    print(epoch, avg_loss)
    
torch.save(model.state_dict(), 'weights.pth')

0 0.545386016368866
1 0.5307008028030396
2 0.5274965763092041
3 0.527568519115448
4 0.5276196599006653
5 0.5273244976997375
6 0.5268319845199585
7 0.5261529088020325
8 0.5259855389595032
9 0.5267547965049744
